In [2]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import os
from datetime import datetime

# Initialize MediaPipe solutions
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

def get_avg_color(region):
    avg_color_per_row = np.average(region, axis=0)
    avg_color = np.average(avg_color_per_row, axis=0)
    return avg_color

def bgr_to_hex(bgr):
    r, g, b = int(bgr[2]), int(bgr[1]), int(bgr[0])
    return "#{:02x}{:02x}{:02x}".format(r, g, b).upper()

def sample_feature_color(frame, landmarks, idx, region_size=6):
    h, w, _ = frame.shape
    x = int(landmarks.landmark[idx].x * w)
    y = int(landmarks.landmark[idx].y * h)
    patch = frame[y-region_size:y+region_size, x-region_size:x+region_size]
    if patch.size == 0:
        return None, (x, y)
    avg_bgr = get_avg_color(patch)
    return avg_bgr, (x, y)

def initialize_csv(filename):
    if not os.path.exists(filename):
        with open(filename, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(['Timestamp', 'Eye_R', 'Eye_G', 'Eye_B', 'Eye_Hex', 
                               'Lips_R', 'Lips_G', 'Lips_B', 'Lips_Hex',
                               'Cheek_R', 'Cheek_G', 'Cheek_B', 'Cheek_Hex',
                               'Hair_R', 'Hair_G', 'Hair_B', 'Hair_Hex'])
    return filename

def append_to_csv(filename, feature_colors):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    row_data = [timestamp]
    
    for feature in ['Eye', 'Lips', 'Cheek', 'Hair']:
        if feature in feature_colors and feature_colors[feature] is not None:
            color_bgr = feature_colors[feature]
            r, g, b = int(color_bgr[2]), int(color_bgr[1]), int(color_bgr[0])
            hex_color = bgr_to_hex(color_bgr)
            row_data.extend([r, g, b, hex_color])
        else:
            row_data.extend([None, None, None, None])
    
    with open(filename, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(row_data)

def start_color_analysis():
    csv_filename = "av-features.csv"
    initialize_csv(csv_filename)
    
    cap = cv2.VideoCapture(0)
    
    # Custom drawing specifications with blue color for mesh
    blue_mesh_spec = mp_drawing.DrawingSpec(color=(250, 17, 219), thickness=1, circle_radius=1)
    
    # MediaPipe FaceMesh configuration
    with mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as face_mesh:
        
        recording = False
        show_mesh = True  # Toggle for showing/hiding the facial mesh
        feature_colors = {}
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
                
            # Convert to RGB for MediaPipe
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Process the image
            results = face_mesh.process(image_rgb)
            
            # Make a copy of the frame for visualization
            display_frame = frame.copy()
            
            if results.multi_face_landmarks:
                face_landmarks = results.multi_face_landmarks[0]
                
                # Draw the facial mesh if enabled
                if show_mesh:
                    # Draw tessellation with custom blue color
                    mp_drawing.draw_landmarks(
                        image=display_frame,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=blue_mesh_spec
                    )
                
                features = {
                    'Eye': 468,   # Left iris center
                    'Lips': 13,   # Lower lip center
                    'Cheek': 205, # Left cheek
                    'Hair': 295   # Eyebrow, in place for hair color
                }
                
                # Reset feature_colors for this frame
                feature_colors = {}
                
                y_offset = 20
                for i, (feature, idx) in enumerate(features.items()):
                    color_bgr, (x, y) = sample_feature_color(frame, face_landmarks, idx)
                    if color_bgr is not None:
                        feature_colors[feature] = color_bgr
                        r, g, b = int(color_bgr[2]), int(color_bgr[1]), int(color_bgr[0])
                        hex_color = bgr_to_hex(color_bgr)
                        
                        # Text display
                        text = f"{feature}: RGB({r},{g},{b}) | {hex_color}"
                        cv2.putText(display_frame, text, (10, 30 + y_offset * i), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                        
                        # Color swatch next to face
                        swatch_x = x + 10
                        swatch_y = y - 10
                        cv2.rectangle(display_frame, (swatch_x, swatch_y), (swatch_x + 40, swatch_y + 20), (b, g, r), -1)
                        cv2.rectangle(display_frame, (swatch_x, swatch_y), (swatch_x + 40, swatch_y + 20), (0, 0, 0), 1)
                        
                        # Highlight feature point with bright red (to stand out against blue mesh)
                        cv2.circle(display_frame, (x, y), 3, (255, 255, 255), -1)
            
            # Display recording status
            if recording:
                cv2.putText(display_frame, "Recording: ON (Press 'r' to stop)", (10, display_frame.shape[0] - 20), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            else:
                cv2.putText(display_frame, "Recording: OFF (Press 'r' to start)", (10, display_frame.shape[0] - 20), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
            # Display CSV status and controls
            cv2.putText(display_frame, f"CSV: {csv_filename}", (10, display_frame.shape[0] - 50), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            cv2.putText(display_frame, "Press 'm' to toggle mesh", (10, display_frame.shape[0] - 80), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
                       
            # Show current frame    
            cv2.imshow('KoreAI Analyzer – Feature Colors', display_frame)
            
            # Handle keyboard input
            key = cv2.waitKey(5) & 0xFF
            if key == 27:  # ESC key
                break
            elif key == ord('r'):  # Toggle recording
                recording = not recording
                if recording:
                    print(f"Recording started. Data will be saved to {csv_filename}")
                else:
                    print(f"Recording stopped. Data saved to {csv_filename}")
            elif key == ord('s') and len(feature_colors) > 0:  # Manual save current frame
                append_to_csv(csv_filename, feature_colors)
                print(f"Snapshot saved to {csv_filename}")
            elif key == ord('m'):  # Toggle mesh visualization
                show_mesh = not show_mesh
                print(f"Face mesh visualization: {'ON' if show_mesh else 'OFF'}")
            
            # If recording and we have detected features, append to CSV
            if recording and results.multi_face_landmarks and len(feature_colors) > 0:
                append_to_csv(csv_filename, feature_colors)
                
    cap.release()
    cv2.destroyAllWindows()
    print(f"Analysis complete. Data saved to {csv_filename}")

if __name__ == "__main__":
    start_color_analysis()

I0000 00:00:1743914660.220063    8471 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1743914660.223508    9850 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743914660.233218    9850 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743914660.236383    9851 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


KeyboardInterrupt: 

In [1]:
import pandas as pd

def csv_to_df(file_name):
    df = pd.read_csv(file_name)[['Eye_R', 'Eye_G', 'Eye_B', 
                                'Lips_R', 'Lips_G', 'Lips_B',
                                'Cheek_R', 'Cheek_G', 'Cheek_B',
                                'Hair_R', 'Hair_G', 'Hair_B']]
    avg = df.mean(axis=0)
    transpose = pd.DataFrame(avg).transpose()
    return transpose

vk = csv_to_df('vk-features.csv')
dd = csv_to_df('dd-features.csv')
sp = csv_to_df('sp-features.csv')
av = csv_to_df('av-features.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'vk-features.csv'

In [14]:
# Undertone

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the data
training = pd.read_csv('../csv/training_celebs.csv')[['undertone', 'Eye_R', 'Eye_G', 'Eye_B', 
                                           'Lips_R', 'Lips_G', 'Lips_B', 'Cheek_R', 'Cheek_G', 
                                           'Cheek_B', 'Hair_R', 'Hair_G', 'Hair_B']]

# Prepare the features and target variable
X = training.drop(columns=['undertone'])
y = training['undertone']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# List of numerical columns (features that are already numeric)
num_columns = ['Eye_R', 'Eye_G', 'Eye_B', 'Lips_R', 'Lips_G', 'Lips_B', 
               'Cheek_R', 'Cheek_G', 'Cheek_B', 'Hair_R', 'Hair_G', 'Hair_B']

from sklearn.linear_model import LogisticRegression

pipeline = Pipeline(steps=[
    ('classifier', LogisticRegression())
])

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
def predict_undertone(features):
    pred = pipeline.predict(features)
    return pred

import joblib

# Save the pipeline to a file
joblib.dump(pipeline, '../models/undertone_classifier3.pkl')

# predict_undertone(av)

# Evaluate the model
y_pred = pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))  

Accuracy: 0.4


/Users/snigdhapodugu/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
# Season

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the data
training = pd.read_csv('../csv/training_celebs.csv')[['season', 'Eye_R', 'Eye_G', 'Eye_B', 
                                           'Lips_R', 'Lips_G', 'Lips_B', 'Cheek_R', 'Cheek_G', 
                                           'Cheek_B', 'Hair_R', 'Hair_G', 'Hair_B']]

# Prepare the features and target variable
X = training.drop(columns=['season'])
y = training['season']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# List of numerical columns (features that are already numeric)
num_columns = ['Eye_R', 'Eye_G', 'Eye_B', 'Lips_R', 'Lips_G', 'Lips_B', 
               'Cheek_R', 'Cheek_G', 'Cheek_B', 'Hair_R', 'Hair_G', 'Hair_B']

from sklearn.neural_network import MLPClassifier

from sklearn.linear_model import LogisticRegression

pipeline = Pipeline(steps=[
    ('classifier', LogisticRegression())
])

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
def predict_season(features):
    pred = pipeline.predict(features)
    return pred

import joblib

# Save the pipeline to a file
joblib.dump(pipeline, '../models/season_classifier3.pkl')

# Evaluate the model
y_pred = pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.2


/Users/snigdhapodugu/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
